<a href="https://colab.research.google.com/github/francoisdoanp/projects/blob/master/DL_HW2_Francois_Doan_Pope_(11178569).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Homework 2

By: Francois Doan-Pope

Student ID: 11178569

---



# Details

For this homework, I have tried three different libraries. The first one (Stellargraph) is the one that was used to get my score on the leaderboard. 

Some other models were tried, but were not featured in the code.


In [ ]:
# General imports

import pandas as pd
import numpy as np
import os

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Used to export prediction file

"""
from google.colab import drive
drive.mount('drive')
"""

In [ ]:
# General data import

# Defining files path

base_url = "https://raw.githubusercontent.com/francoisdoanp/homeworks/master/Deep%20Learning%20HW2/"

# Importing files

tr = pd.read_csv(base_url + "train.csv")
tex = pd.read_csv(base_url + "text.csv")
reference = pd.read_csv(base_url + "reference.csv", names=["target", "source"], skiprows=1)
t = pd.read_csv(base_url + "test.csv")

# Stellargraph

## Imports & Installs

In [ ]:
!pip install stellargraph

In [ ]:
import networkx as nx


# StellarGraph
import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator, GraphSAGENodeGenerator
from stellargraph.layer import GCN, GraphSAGE, GAT

# Text libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection


# Keras
from keras.utils import to_categorical
from tensorflow.keras import layers, optimizers, losses, metrics, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.models import Sequential


## Data Preparation

In [ ]:
# Joining dataframes

data = pd.merge(tr, tex, left_on='id', right_on='id', how='left')
test = pd.merge(t, tex, left_on='id', right_on='id', how='left')

# Building vocab

vectorizer = CountVectorizer()
vectorizer.fit(tex.title)

# Transform the data and test DF

bowd = vectorizer.transform(data.title).toarray()
bowt = vectorizer.transform(test.title).toarray()

# Concatenating BoW with original DF

data = pd.concat([tr, pd.DataFrame(bowd)], axis=1)
data.set_index('id', inplace=True)
test = pd.concat([t, pd.DataFrame(bowt)], axis=1)
test.set_index('id', inplace=True)

# Creating one graph

df_all = pd.concat([data,test]).drop('label', axis=1)
df_all_lab = pd.concat([data,test])

# Train / test mask

train_mask = (np.isnan(df_all_lab['label']) == False)
test_mask = (np.isnan(df_all_lab['label'] == True)) 

# Labels to one-hot

Y_train_all = to_categorical(data['label'])

In [ ]:
# Splitting in train and valid

train_data, valid_data = model_selection.train_test_split(data, train_size=0.95)

# Creating labels

Y_train = to_categorical(train_data['label'])
Y_valid = to_categorical(valid_data['label'])

# Features

X_train = train_data.drop('label', axis=1)
X_valid = valid_data.drop('label', axis=1)

In [ ]:
# Creating graph

graph = sg.StellarDiGraph(nodes={"paper": df_all}, edges={"cites": reference})

## GCN Model

In [ ]:
generator = FullBatchNodeGenerator(graph, method="gcn")

Using GCN (local pooling) filters...


In [ ]:
train_gen = generator.flow(X_train.index, Y_train)
val_gen = generator.flow(X_valid.index, Y_valid)

# Building model
gcn = GCN(layer_sizes=[128,128], activations=["relu", "relu"], generator=generator, dropout=0.3)
x_in, x_out = gcn.build()
predictions = layers.Dense(units=Y_train.shape[1], activation='softmax')(x_out)

modelGCN = Model(inputs=x_in, outputs=predictions)
modelGCN.compile(optimizer=optimizers.Adam(lr=0.001),
                 loss='categorical_crossentropy',
                 metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

if not os.path.isdir("logs"):
    os.makedirs("logs")
es_callback = EarlyStopping(
    monitor="val_acc", patience=50
)  # patience is the number of epochs to wait before early stopping in case of no further improvement
mc_callback = ModelCheckpoint(
    "logs/best_model.h5", monitor="val_acc", save_best_only=True, save_weights_only=True
)

In [ ]:
historyGCN = modelGCN.fit_generator(
    train_gen,
    epochs=200,
    validation_data=val_gen,
    verbose=2,
    shuffle=False,  # this should be False, since shuffling data means shuffling the whole graph
    callbacks=[es_callback, mc_callback],
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/200
1/1 - 3s - loss: 1.6087 - acc: 0.2132 - val_loss: 1.6009 - val_acc: 0.4366
Epoch 2/200
1/1 - 3s - loss: 1.5999 - acc: 0.3942 - val_loss: 1.5926 - val_acc: 0.5133
Epoch 3/200
1/1 - 3s - loss: 1.5904 - acc: 0.4851 - val_loss: 1.5827 - val_acc: 0.5790
Epoch 4/200
1/1 - 3s - loss: 1.5795 - acc: 0.5499 - val_loss: 1.5706 - val_acc: 0.6072
Epoch 5/200
1/1 - 3s - loss: 1.5666 - acc: 0.5792 - val_loss: 1.5564 - val_acc: 0.6213
Epoch 6/200
1/1 - 3s - loss: 1.5515 - acc: 0.5984 - val_loss: 1.5399 - val_acc: 0.6322
Epoch 7/200
1/1 - 3s - loss: 1.5342 - acc: 0.6119 - val_loss: 1.5212 - val_acc: 0.6385
Epoch 8/200
1/1 - 3s - loss: 1.5140 - acc: 0.6269 - val_loss: 1.5003 - val_acc: 0.6479
Epoch 9/200
1/1 - 3s - loss: 1.4924 - acc: 0.6360 - val_loss: 1.4773 - val_acc: 0.6573
Epoch 10/200
1/1 - 2s - loss: 1.4680 - acc: 0.6441 - val_loss: 1.4521 - val_acc: 0.6620
Epoch 11/200
1/1 - 2s - loss: 1.4418 - acc: 0.6509 - 

In [ ]:
# Predictions based on "test" set

test_gen_gcn = generator.flow(test.index)
modelGCN.load_weights("logs/best_model.h5")
pred_gcn = modelGCN.predict_generator(test_gen_gcn)

final_pred_gcn = np.argmax(pred_gcn.squeeze(), axis=1)

pred_out_gcn = test
pred_out_gcn['label'] = final_pred_gcn
pred_out_gcn = pred_out_gcn.loc[:,['label']]

Instructions for updating:
Please use Model.predict, which supports generators.


In [ ]:
# Exporting

pred_out_gcn.to_csv('pred_gcn2.csv')
!cp pred_gcn2.csv "drive/My Drive/deep learning/"

## GraphSage

In [ ]:
batch_size = 50
num_samples = [10,5]

generatorSAGE = GraphSAGENodeGenerator(graph, batch_size, num_samples)

trainSAGE_gen = generatorSAGE.flow(X_train.index, Y_train, shuffle=True)

graphsage_model = GraphSAGE(layer_sizes=[32,32], generator=generatorSAGE, bias=True, dropout=0.5)

xs_in, xs_out = graphsage_model.build()
predictionSAGE = layers.Dense(units=Y_train.shape[1], activation='softmax')(xs_out)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: The 'build' method is deprecated, use 'in_out_tensors' instead.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
modelSAGE = Model(xs_in, predictionSAGE)
modelSAGE.compile(optimizer=optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['acc'])


In [ ]:
if not os.path.isdir("logs"):
    os.makedirs("logs")
es_callback = EarlyStopping(
    monitor="val_acc", patience=20
)  # patience is the number of epochs to wait before early stopping in case of no further improvement
mc_callback = ModelCheckpoint(
    "logs/best_model.h5", monitor="val_acc", save_best_only=True, save_weights_only=True
)

validSAGE_gen = generatorSAGE.flow(X_valid.index, Y_valid)
historySAGE = modelSAGE.fit_generator(trainSAGE_gen, epochs=75, validation_data=validSAGE_gen, verbose=2, shuffle=False, callbacks=[es_callback, mc_callback])

## GAT

In [ ]:
generatorGAT = FullBatchNodeGenerator(graph, method="gat")

train_gat = generatorGAT.flow(X_train.index, Y_train)
val_gat =  generatorGAT.flow(X_valid.index, Y_valid)

gat = GAT(
    layer_sizes=[256,256,256, Y_train.shape[1]],
    activations=["elu", "elu", "elu", "softmax"],
    attn_heads=6,
    generator=generatorGAT,
    in_dropout=0.4,
    attn_dropout=0.5,
    normalize=None,
)

xga_in, xga_pred = gat.in_out_tensors()

In [ ]:
modelGAT = Model(inputs=xga_in, outputs=xga_pred)
modelGAT.compile(optimizer=optimizers.Adam(lr=0.005),
                 loss=losses.categorical_crossentropy,
                 metrics=['acc'],)

In [ ]:
historyGAT = modelGAT.fit(train_gat, epochs=150, validation_data=val_gat, verbose=2, shuffle=False, callbacks=[es_callback, mc_callback],)

# Pytorch Geometric

For some reason, I could not get pytorch geometric to obtain results comparable to Stellargraph using the same models. This is too bad as it is much faster than Stellargraph, and SplineCNN showed great promise.

## Imports & Installs

In [ ]:
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-geometric

In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import SplineConv, GATConv
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import degree

## Data Preparation

In [ ]:
# Adding edge weight column (some models require edge attributes)

reference['weight'] = 1

In [ ]:
# Joining dataframes

data = pd.merge(tr, tex, left_on='id', right_on='id', how='left')
test_pd = pd.merge(t, tex, left_on='id', right_on='id', how='left')

# Building vocab

vectorizer = CountVectorizer()
vectorizer.fit(tex.title)

# Transform the data and test DF

bowd = vectorizer.transform(data.title).toarray()
bowt = vectorizer.transform(test_pd.title).toarray()

# Concatenating BoW with original DF

data = pd.concat([tr, pd.DataFrame(bowd)], axis=1)
data.set_index('id', inplace=True)
test_pd = pd.concat([t, pd.DataFrame(bowt)], axis=1)
test_pd.set_index('id', inplace=True)

# Creating one DF
df_all = pd.concat([data,test_pd]).drop('label', axis=1)
df_all_lab = pd.concat([data,test_pd])

# Creating masks
"""
This is my crude way of creating masks when we have unknowns (data to predict).
If you could provide a cleaner way for me to do so, I would be very happy, because this is barely acceptable, lol.
Although now that I am writing this, I am thinking that we can simply leave out the "test" set and still provide the full
graph to the model... 
"""
# 80/20 valid split
df_all_lab['randNumCol'] = np.random.randint(1, 21, df_all_lab.shape[0])

test_maskdl = torch.tensor((np.isnan(df_all_lab['label']) == True), dtype=torch.bool)
train_maskdl = torch.tensor((np.isnan(df_all_lab['label']) == False) & (df_all_lab.randNumCol < 19), dtype=torch.bool)
valid_maskdl = torch.tensor((np.isnan(df_all_lab['label']) == False) & (df_all_lab.randNumCol == 20), dtype=torch.bool)
df_all_lab = df_all_lab.drop(['randNumCol'], axis=1)


# Since we cannot have NA's, I replaced by 5 (which will not be used at output will be 5 labels)
df_all_lab['label'].fillna(5, inplace=True)

In [ ]:
# Converting to torch tensor

edg_temp = torch.tensor(reference['weight'].values, dtype=torch.float)
edge_attrdl = edg_temp.unsqueeze(1)
edge_indexdl = torch.tensor(reference.drop('weight', axis=1).values, dtype=torch.long)
xdl = torch.tensor(df_all_lab.drop('label',axis=1).values, dtype=torch.float)
ydl = torch.tensor(df_all_lab['label'].values, dtype=torch.long)

In [ ]:
# Compute normalized degree (not necessary)
te = T.TargetIndegree()

# Creating data for pytorch geo
temp = Data(x=xdl, edge_index=edge_indexdl.t().contiguous(), y=ydl)

data_dl = te(temp)

# Loading masks in data
data_dl.train_mask = train_maskdl
data_dl.valid_mask = valid_maskdl
data_dl.test_mask = test_maskdl

In [ ]:
# Model independent 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data_dl.train_mask], data_dl.y[data_dl.train_mask]).backward()
    optimizer.step()

def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data_dl('train_mask', 'valid_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data_dl.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


## SplineCNN

In [ ]:
# Defining model

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SplineConv(data_dl.num_features, 128, dim=1, kernel_size=2)
        self.conv2 = SplineConv(128, 5, dim=1, kernel_size=2)

    def forward(self):
        x, edge_index, edge_attr = data_dl.x, data_dl.edge_index, data_dl.edge_attr
        x = F.dropout(x, training=self.training)
        x = F.elu(self.conv1(x, edge_index, edge_attr))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        return F.log_softmax(x, dim=1)

In [ ]:
model, data = Net().to(device), data_dl.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

## GAT

In [ ]:
# Defining model

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GATConv(data_dl.num_features, 256, heads=6), dropout=0.6)
        self.conv2 = GATConv(
            256 * 6, 5, heads=1, concat=True, dropout=0.6)
        
    def forward(self):
        x = F.dropout(data_dl.x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, data_dl.edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, data_dl.edge_index)
        return F.log_softmax(x, dim=1)

In [ ]:
model, data = Net().to(device), data_dl.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

## Training & Predictions


In [ ]:
# Training

for epoch in range(1, 16):
    train()
    log = 'Epoch: {:03d}, Train: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, *test()))

In [ ]:
# Predictions

pred = model()[data_dl.test_mask]
final_pred = np.argmax(pred.data.cpu(), axis=1).numpy()

pred_out_gcn = test_pd
pred_out_gcn['label'] = final_pred
pred_out_gcn = pred_out_gcn.loc[:,['label']]

# Spektral

This is the first library I tried. I strongly prefer the other two.

## Imports & Installs

In [ ]:
# Installs

! apt install graphviz libgraphviz-dev libcgraph6
! pip install spektral

In [ ]:
# Imports

import math
import numpy.ma as ma

# Graph reprensentations
from spektral.layers import GraphConv, ChebConv, ARMAConv, GraphAttention
from spektral import utils
from spektral.utils.misc import add_eye

# Keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dropout, Embedding, LSTM, SpatialDropout1D, Dense, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# Utils
from sklearn.model_selection import train_test_split
from scipy import sparse
import networkx as nx

# Language processing
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk import word_tokenize

import random


## Data Preparation

In [ ]:
# Removing stopwords 

text['title'] = text['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Getting maximum length for padding

max_seq_length = text['title'].map(lambda x: len(x.split())).max()

# Building vocab

tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(text['title'].values)

# Get number of words

num_words = len(tokenizer.word_index)+1

train_df = pd.merge(tr, text, left_on='id', right_on='id', how='left')
test_df = pd.merge(te, text, left_on='id', right_on='id', how='left')

In [ ]:
# Preparing data for Spektral.
# Note that at this point, I was a bit overwhelmed, resulting in weird codes

full_data = pd.concat([train_df, test_df])
full_data = full_data.sort_values(by=['id']).reset_index()
full_data = full_data.drop(['index', 'id'], axis=1)
full_data['label'].fillna(-1, inplace=True)

In [ ]:
# Creating masks

# 80/20 valid split
"""
This method of splitting was also used in Pyotrch Geometric. Once again, let me know what
you would have done instead!
"""
full_data['randNumCol'] = np.random.randint(1, 6, full_data.shape[0])

test_mask = (full_data['label'] == -1)
train_mask = (full_data['label'] != -1) & (full_data.randNumCol < 5)
valid_mask = (full_data['label'] != -1) & (full_data.randNumCol == 5)    

full_data = full_data.drop(['randNumCol'], axis=1)

# To one-hot
Y_g = pd.get_dummies(full_data['label'].values).drop(-1, axis=1).to_numpy()

In [ ]:
# Creating sparse adjacency matrix

size = len(text)
adj_matrix = [[0]*size for _ in range(size)]

for i, id in reference.values.tolist():
  adj_matrix[i][id] = 1

adj = sparse.csr_matrix(np.asarray(adj_matrix))
print(adj.shape)

In [ ]:
# Tokenizing

tokenizer = Tokenizer()
tokenizer.fit_on_texts(full_data['title'].values)

X_tok = tokenizer.texts_to_sequences(full_data['title'].values)

vocab_size = len(tokenizer.word_index)+1
X_tok = pad_sequences(X_tok, padding='post', maxlen=max_seq_length)

## GCN

In [ ]:
# Preprocess

A_filter = utils.localpooling_filter(adj).astype('f4')

# Model's Definitions
N = adj.shape[0]
F = X_tok.shape[-1]
n_class = Y_g.shape[-1]
dropout = 0.4
lr = 0.001
epochs = 1000
patience = 50
regularizer = 0.00005

# Building model
X_input = Input(shape=(F, ))
A_input = Input((N, ), sparse=True)

dropout_1 = Dropout(0.5)(X_input)
graph_conv1 = GraphConv(36, activation='relu', use_bias=False, kernel_regularizer=l2(regularizer))([dropout_1, A_input])
dropout_2 = Dropout(dropout)(graph_conv1)
graph_conv2 = GraphConv(n_class, activation='softmax', use_bias=False, kernel_regularizer=l2(regularizer))([dropout_2, A_input])


model = Model(inputs=[X_input, A_input], outputs=graph_conv2)
optimizer = Adam(learning_rate=lr)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', weighted_metrics=['acc'])


In [ ]:
# Fitting model

validation_data = ([X_tok, A_filter], Y_g, valid_mask)
model.fit([X_tok, A_filter],
          Y_g,
          sample_weight=train_mask,
          epochs = 1000,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False,
          callbacks=[
              EarlyStopping(patience=patience,  restore_best_weights=True)]
          )

In [ ]:
# Predictions
graph_pred = model.predict_on_batch([X_tok, A_filter])

# Building final prediction table
final_pred2 = np.argmax(graph_pred[np.array(test_mask)], axis=1)
print(final_pred2.shape)

pred_out2 = te

pred_out2['label'] = final_pred2


## Chebnets

In [ ]:
# Preprocess
cheb_fltr = ChebConv.preprocess(adj).astype('f4')

# Definitions

N = adj.shape[0]
F = X_tok.shape[-1]
n_class = Y_g.shape[-1]
dropout = 0.4
lr = 0.001
epochs = 1000
patience = 20
regularizer = 0.00005

k = 2

X_input = Input(shape=(F, ))
A_input = Input((N, ), sparse=True)


cheb_drop_1 = Dropout(dropout)(X_input)
cheb_conv1 = ChebConv(64, activation='relu', use_bias=False, K=k)([cheb_drop_1, A_input])
cheb_drop_2 = Dropout(dropout)(cheb_conv1)
cheb_conv2 = ChebConv(64, activation='relu', use_bias=False, K=k)([cheb_drop_2, A_input])
cheb_drop_3 = Dropout(dropout)(cheb_conv2)
cheb_conv3 = ChebConv(n_class, activation='softmax', use_bias=False, K=k)([cheb_drop_3, A_input])

model2 = Model(inputs=[X_input, A_input], outputs=cheb_conv3)
optimizer = Adam(learning_rate=lr)
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', weighted_metrics=['acc'])

In [ ]:
# Fitting model

validation_data = ([X_tok, cheb_fltr], Y_g, valid_mask)

model2.fit([X_tok, cheb_fltr],
          Y_g,
          sample_weight=train_mask,
          epochs = epochs,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False,
          callbacks=[
              EarlyStopping(patience=patience,  restore_best_weights=True)]
          )

In [ ]:
# Predict

# Computing predictions
cheb_pred = model2.predict_on_batch([X_g, cheb_fltr])

# Building final prediction table
final_pred = np.argmax(cheb_pred[np.array(test_mask)], axis=1)
print(final_pred.shape)

pred_out = te

pred_out['label'] = final_pred

## GAT

In [ ]:
# Model definition

A = add_eye(adj).toarray()

X_in = Input(shape=(F, ))
A_in = Input((N, ), sparse=False)

dropout_1 = Dropout(0.5)(X_in)
graph_attention_1 = GraphAttention(8,
                                   attn_heads=8,
                                   concat_heads=True,
                                   dropout_rate=0.5,
                                   activation='elu',
                                   kernel_regularizer=l2(regularizer),
                                   attn_kernel_regularizer=l2(regularizer)
                                   )([dropout_1, A_in])
dropout_2 = Dropout(0.5)(graph_attention_1)
graph_attention_2 = GraphAttention(n_class,
                                   attn_heads=1,
                                   concat_heads=False,
                                   dropout_rate=0.5,
                                   activation='softmax',
                                   kernel_regularizer=l2(regularizer),
                                   attn_kernel_regularizer=l2(regularizer)
                                   )([dropout_2, A_in])

GAT_model = Model(inputs=[X_in, A_in], outputs=graph_attention_2)
optimizer = Adam(lr=0.001)
GAT_model.compile(optimizer=optimizer, loss='categorical_crossentropy', weighted_metrics=['acc'])
GAT_model.summary()

In [ ]:
# Fitting

validation_data = ([X_g, A], Y_g, valid_mask)
GAT_model.fit([X_g, A], Y_g, sample_weight=train_mask, epochs=1000, batch_size=N, validation_data=validation_data, shuffle=False, callbacks=[EarlyStopping(patience=20, restore_best_weights=True)])

## Graph Conv with ARMA

In [ ]:
# Model definition

ARMA_channels = 16
share_weights = True
ARMA_lr = 0.01
ARMA_epochs = 1000
ARMA_patience = 100

ARMA_fltr = ARMAConv.preprocess(adj).astype('f4')

ARMA1 = ARMAConv(ARMA_channels, iterations=1, order=2, share_weights=True, dropout_rate=0.75, activation='elu', gcn_activation='elu',
                 kernel_regularizer=l2(regularizer))([X_input, A_input])
ARMA_dropout = Dropout(0.5)(ARMA1)
ARMA2 = ARMAConv(n_class, iterations=1, order=1, share_weights=True, dropout_rate=0.75, activation='softmax', gcn_activation=None,
                       kernel_regularizer=l2(regularizer))([ARMA_dropout, A_input])

model3 = Model(inputs=[X_input, A_input], outputs=ARMA2)
optimizer = Adam(learning_rate=lr)
model3.compile(optimizer=optimizer, loss='categorical_crossentropy', weighted_metrics=['acc'])

In [ ]:
# Fitting

model3.fit([X_g, A_filter],
          Y_g,
          sample_weight=train_mask,
          epochs = ARMA_epochs,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False,
          callbacks=[
              EarlyStopping(patience=ARMA_patience,  restore_best_weights=True)]
          )